In [2]:
import numpy as np
import pandas as pd
import altair as alt 

In [3]:
df = pd.read_csv('./data/target.csv')
df.dates = pd.to_datetime(df.dates)
# let's add a column that it will be used in the legend following this hack: https://github.com/altair-viz/altair/issues/984
df['legend'] = 'Data'
df.head()

,dates,target,legend
0,2021-12-26,54.0,Data
1,2022-01-02,212.0,Data
2,2022-01-09,236.0,Data
3,2022-01-16,174.0,Data
4,2022-01-23,115.0,Data


To plot this data in Altair it's necessary to have the same name for the predictions of different models (represent by the columns), and one column that indicates for which model the predictions belong ('model' in the case below). If you have a dataset strcuture by column, take look at the pd.melt function (https://pandas.pydata.org/docs/reference/api/pandas.melt.html)

In [4]:
df_for = pd.read_csv('./data/forecasts.csv')
df_for.dates = pd.to_datetime(df_for.dates)
df_for

,dates,model,predictions,lower,upper
0,2021-12-26,RF,224.030000,0.000000,657.151984
1,2022-01-02,RF,311.150000,0.000000,926.336532
2,2022-01-09,RF,989.630000,0.000000,2139.728589
3,2022-01-16,RF,1550.550000,104.709219,2996.390781
4,2022-01-23,RF,368.890000,0.000000,2020.111839
...,...,...,...,...,...
315,2023-06-04,DL - cluster,304.602886,158.264750,426.872812
316,2023-06-11,DL - cluster,346.068766,160.559812,536.159720
317,2023-06-18,DL - cluster,343.693966,191.552979,543.737511
318,2023-06-25,DL - cluster,363.179827,180.028784,503.317822


Creating the plot in altair:

The plots in altair are a lot of layers concatenated together. Note that except the `data` element, all the others are based on the `base` element, where it's loaded the `df_for` dataset.

In [11]:
# here is loaded the element that allows the selection by the mouse
highlight = alt.selection_point(on='mouseover', value = 'DL', fields=['model'], nearest=True)

width = 375 # width of the plots

# here is loaded the data points (black)
data = alt.Chart(df).mark_circle(size = 60).encode(
    x='dates:T',
    y='target:Q',
    color=alt.value('black'),
    opacity=alt.Opacity('legend', legend=alt.Legend(title=None)),

    #size = alt.value(3)
    tooltip = 'target:Q'
).properties(
    width=width
)


# here is created the base element for the time series 
base = alt.Chart(df_for, title="Median Forecast of dengue").encode(
   x=alt.X('dates:T').title('Dates'),
    y=alt.Y('predictions:Q').title('New cases'),
    color='model:N'
).add_params(
    highlight
).properties(
    width=width
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_params(
    highlight
)


# here we create the multine plot and use the alt.condition to highlight only one curve
lines = base.mark_line().encode(
    #size=alt.condition(~highlight, alt.value(1), alt.value(3))
    color=alt.condition(highlight, alt.Color('model:N'), alt.value('lightgray')),
    tooltip = ['model:N', 'predictions']
    
)

# here we define the plot of the right figure
timeseries = alt.Chart(df_for, title="Median and 95% confidence interval").mark_line().encode(
    x=alt.X('dates:T').title('Dates'),
    y=alt.Y('predictions:Q').title('New cases'),
    color=alt.Color('model:N'),
).transform_filter(
    highlight # this function transform filter will just filter the element 
    #in hightlight from the column model N of the df_for (defined in the base element)
)

# here we create the area that represent the confidence interval of the predicitions
timeseries_conf = (timeseries.mark_area(
    opacity=0.5,
).encode(
    x='dates:T',
    y='lower:Q',
    y2='upper:Q'
)
.transform_filter(
    highlight
))

# here we concatenate the layers, the + put one layer above the other
# the | put them syde by syde (as columns), and & put them side by side as lines
final = points + lines + data | timeseries + timeseries_conf + data

final

alt.HConcatChart(...)

We can save this data as html using: 

In [19]:
final.save('./files_html/forecast_dengue.html')

In [18]:
# Saving as png it's not working with altair 5.0
final.save('./figures/forecast_dengue.png')

NoMatchingVersions: No matches for version='5.15.1' among ['4.0.2', '4.8.1', '4.17.0'].
Often this can be fixed by updating altair_viewer:
    pip install -U altair_viewer

In [20]:
final.save('./files_html/forecast_dengue.json')